<a href="https://colab.research.google.com/github/ImaneElAssari/LoanPredictionModel/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Etape 1` : **Data Cleaning**




In [29]:
#importer les packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
#pour la visualisation
import matplotlib.pyplot as pl 
import seaborn as sns
import pickle  #permet  de generer le modele a deployer

In [3]:
#Lire la base de donnes 
df=pd.read_csv('/content/train_u6lujuX_CVtuZ9i.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [4]:
#Pour afficher toute la base de donnes si on execute df
pd.set_option('display.max_rows',df.shape[0]+1)

**Remarque** :  le **Nna** indique l'existance des données manquantes





In [6]:
#On veut afficher juste les 10  lignes 
pd.set_option('display.max_rows',10)
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


Maintenant l'execution de 'df' nous a affiché seulement 10 lignes : les 5 premiers, et les 5 derniers

In [5]:
# afficher les valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


Cette dataset contient 614 enregistrement



In [6]:
df.isnull().sum().sort_values(ascending=False)

Credit_History       50
Self_Employed        32
LoanAmount           22
Dependents           15
Loan_Amount_Term     14
                     ..
Property_Area         0
CoapplicantIncome     0
ApplicantIncome       0
Education             0
Loan_ID               0
Length: 13, dtype: int64

On veut savoir le nomre de valeurs manquantes pour chaque attribut.
On cherche dans la base de donnees les cases ou il y a une valeur manquante, on somme toutes ces valeur et puis on applique la methode sort


In [7]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


count = somme des valeurs de chaque enregistrement


mean = moyenne 


std = ecart-type pour 1er quartile (25%), 2eme(50%), et le 3 eme(75%)

Determination des valeurs erronees: Si l'on trouve par exemple une valeur qui est inferieur au min (=150 dans ce cas)  alors ca sera une valeur erronee(**outlayer**=valeur qui est a l'ecart des autres). Meme chose pour le max

In [9]:
#Pour les valeurs categoriques
df.describe(include='O')

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
count,614,601,611,599,614,582,614,614
unique,614,2,2,4,2,2,3,2
top,LP001319,Male,Yes,0,Graduate,No,Semiurban,Y
freq,1,489,398,345,480,500,233,422


In [10]:
#renseigner les valeurs manquantes
#On cree 2 listes(ce sont des listes contenant d'autres listes)
cat_data=[]
num_data=[]
for i,c in enumerate(df.dtypes):#on va classer les attributs selon leur type
   if c==object:
     cat_data.append(df.iloc[:,i])#iloc permet de selctionner toute la colonne
   else:
     num_data.append(df.iloc[:,i])
cat_data

[0      LP001002
 1      LP001003
 2      LP001005
 3      LP001006
 4      LP001008
          ...   
 609    LP002978
 610    LP002979
 611    LP002983
 612    LP002984
 613    LP002990
 Name: Loan_ID, Length: 614, dtype: object, 0        Male
 1        Male
 2        Male
 3        Male
 4        Male
         ...  
 609    Female
 610      Male
 611      Male
 612      Male
 613    Female
 Name: Gender, Length: 614, dtype: object, 0       No
 1      Yes
 2      Yes
 3      Yes
 4       No
       ... 
 609     No
 610    Yes
 611    Yes
 612    Yes
 613     No
 Name: Married, Length: 614, dtype: object, 0       0
 1       1
 2       0
 3       0
 4       0
        ..
 609     0
 610    3+
 611     1
 612     2
 613     0
 Name: Dependents, Length: 614, dtype: object, 0          Graduate
 1          Graduate
 2          Graduate
 3      Not Graduate
 4          Graduate
            ...     
 609        Graduate
 610        Graduate
 611        Graduate
 612        Graduate
 613       

**Remarque :** On peut pas manipuler les deux listes cat_data et num_data, on doit les transformer a des BDD


In [11]:
#transformer nos listes a des BDDs

cat_data=pd.DataFrame(cat_data)
cat_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613
Loan_ID,LP001002,LP001003,LP001005,LP001006,LP001008,LP001011,LP001013,LP001014,LP001018,LP001020,LP001024,LP001027,LP001028,LP001029,LP001030,LP001032,LP001034,LP001036,LP001038,LP001041,LP001043,LP001046,LP001047,LP001050,LP001052,LP001066,LP001068,LP001073,LP001086,LP001087,LP001091,LP001095,LP001097,LP001098,LP001100,LP001106,LP001109,LP001112,LP001114,LP001116,...,LP002863,LP002868,LP002872,LP002874,LP002877,LP002888,LP002892,LP002893,LP002894,LP002898,LP002911,LP002912,LP002916,LP002917,LP002925,LP002926,LP002928,LP002931,LP002933,LP002936,LP002938,LP002940,LP002941,LP002943,LP002945,LP002948,LP002949,LP002950,LP002953,LP002958,LP002959,LP002960,LP002961,LP002964,LP002974,LP002978,LP002979,LP002983,LP002984,LP002990
Gender,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Male,Female,Male,Male,Male,Male,Male,NaN,Male,Male,Male,Male,Male,Female,Male,Male,Male,Male,Male,Male,Male,Female,Male,Male,...,Male,Male,NaN,Male,Male,Male,Male,Male,Female,Male,Male,Male,Male,Female,NaN,Male,Male,Male,NaN,Male,Male,Male,Male,Male,Male,Male,Female,Male,Male,Male,Female,Male,Male,Male,Male,Female,Male,Male,Male,Female
Married,No,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,Yes,No,No,Yes,No,Yes,Yes,Yes,No,No,...,Yes,Yes,Yes,No,Yes,No,Yes,No,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,No
Dependents,0,1,0,0,0,2,0,3+,2,1,2,2,2,0,2,0,1,0,0,0,0,1,0,2,1,0,0,2,0,2,1,0,1,0,3+,0,0,0,0,0,...,3+,2,0,0,1,0,2,0,0,1,1,1,0,0,0,2,0,2,3+,0,0,0,2,NaN,0,2,3+,0,3+,0,1,0,1,2,0,0,3+,1,2,0
Education,Graduate,Graduate,Graduate,Not Graduate,Graduate,Graduate,Not Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Not Graduate,Graduate,Not Graduate,Graduate,Not Graduate,Graduate,Not Graduate,Not Graduate,Graduate,Graduate,Graduate,Not Graduate,Not Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Not Graduate,...,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Not Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Not Graduate,Not Graduate,Graduate,Graduate,Graduate,Graduate,Not Graduate,Graduate,Graduate,Graduate,Not Graduate,Graduate,Not Graduate,Graduate,Graduate,Graduate,Graduate,Graduate,Graduate
Self_Employed,No,No,Yes,No,No,Yes,No,No,No,No,No,NaN,No,No,No,No,No,No,No,NaN,No,No,No,No,NaN,Yes,No,No,No,NaN,NaN,No,Yes,No,No,No,No,No,No,No,...,No,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,Yes,No,Yes,Yes,No,Yes,No,Yes,No,Yes,No,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,Yes
Property_Area,Urban,Rural,Urban,Urban,Urban,Urban,Urban,Semiurban,Urban,Semiurban,Urban,Urban,Urban,Rural,Urban,Urban,Urban,Urban,Rural,Urban,Urban,Urban,Semiurban,Rural,Semiurban,Semiurban,Semiurban,Urban,Urban,Semiurban,Urban,Urban,Rural,Semiurban,Rural,Urban,Urban,Semiurban,Urban,Semiurban,...,Semiurban,Urban,Semiurban,Urban,Rural,Urban,Semiurban,Urban,Semiurban,Rural,Rural,Rural,Urban,Semiurban,Semiurban,Semiurban,Semiurban,Semiurban,Semiurban,Rural,Urban,Rural,Rural,Semiurban,Rural,Urban,Urban,Rural,Urban,Rural,Semiurban,Urban,Semiurban,Rural,Rural,Rural,Rural,Urban,Urban,Semiurban
Loan_Status,Y,N,Y,Y,Y,Y,Y,N,Y,N,Y,Y,Y,N,Y,Y,Y,N,N,Y,N,Y,N,N,N,Y,Y,Y,N,Y,N,N,N,Y,N,Y,N,Y,Y,Y,...,N,Y,N,Y,Y,Y,Y,N,Y,N,N,N,Y,Y,Y,N,Y,N,Y,Y,Y,Y,N,N,Y,Y,N,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,N


La base donnee qu'on a eu est inversé car on a applique la fonction DataFrame a la liste et donc elle a pris les indices comme des attributs. Pour resoudre ce problme on doit utiliser la transposé

In [14]:
cat_data=pd.DataFrame(cat_data).transpose()
cat_data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,Urban,Y
4,LP001008,Male,No,0,Graduate,No,Urban,Y
...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,Urban,Y


In [24]:
#de meme pour la liste des donnees numeriques num_data
num_data=pd.DataFrame(num_data).transpose()
num_data

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,5849.0,0.0,NaN,360.0,1.0
1,4583.0,1508.0,128.0,360.0,1.0
2,3000.0,0.0,66.0,360.0,1.0
3,2583.0,2358.0,120.0,360.0,1.0
4,6000.0,0.0,141.0,360.0,1.0
...,...,...,...,...,...
609,2900.0,0.0,71.0,360.0,1.0
610,4106.0,0.0,40.0,180.0,1.0
611,8072.0,240.0,253.0,360.0,1.0
612,7583.0,0.0,187.0,360.0,1.0


**Renseigner les valeurs manquantes:**

**Remarque:** Au cas ou la valeur manquente ne perturbe pas le modele ou si on a assez de valeurs dans la bdd, on peut ne pas la renseigner 

  Pour les valeurs catégoriques : on va remplacer la valeur  manquante par la valeur qui se repete le plus (qui a la haute frequence) exemple gender:on a la categorie 'Male' qui se repete le plus donc a chaque fois on trouve une valeur manquante dans le gender on va la remplacer par 'Male'.Ceci va nous garantir un model non biaisé.

In [21]:
#Cas de valeurs categoriques
cat_data=cat_data.apply(lambda x:x.fillna(x.value_counts().index[0]))#on a remplacer les NNa par la valeur la plus dominante pour chaque attribut
cat_data.isnull().sum().any()

False

In [19]:
#explication: x.value_counts().index[0] 
#value_counts retourne une liste  contenant  le nombre d'occurence de chaque valeur (trie d'une maniere decroissante)
# index[0] prend la valeur la plus frequente
cat_data['Education'].value_counts()

Graduate        480
Not Graduate    134
Name: Education, dtype: int64

Pour les valeurs numeriques: on va renseigner la valeur manquante qui se trouve dans un rang (n) comme similaire a la valeure ayant le rang (n-1)

In [26]:
#Cas de valeurs numeriques
num_data.fillna(method='bfill',inplace=True)
num_data.isnull().sum().any()#retourne un boolean qui verifie s'il reste emcore des valeurs manquantes

False

bdill : permet de remplacer directement les valeurs manquantes par la valeur precedente de la meme colonne

In [27]:
num_data

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,5849.0,0.0,128.0,360.0,1.0
1,4583.0,1508.0,128.0,360.0,1.0
2,3000.0,0.0,66.0,360.0,1.0
3,2583.0,2358.0,120.0,360.0,1.0
4,6000.0,0.0,141.0,360.0,1.0
...,...,...,...,...,...
609,2900.0,0.0,71.0,360.0,1.0
610,4106.0,0.0,40.0,180.0,1.0
611,8072.0,240.0,253.0,360.0,1.0
612,7583.0,0.0,187.0,360.0,1.0


La variable target '**Loan_Status**' est de type string contenant des 'Y/N', on veut les remplacer par 0 et 
1

In [28]:
#Transformer la colonne target
target_value={'Y':1,'N':0}
target=cat_data['Loan_Status']
cat_data.drop('Loan_Status',axis=1,inplace=True)
target=target.map(target_value)#mapage entre Y/1 N/0
target

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

On veut maintennat remplacer le reste des valeurs categoriques par des 0 et des 1 d'une maniere automatique. Pour se faire on aura besoin de LabelEncoder qui se trouve dans sklearn

In [30]:
#instantiation de LabelEncoder
le=LabelEncoder()
for i in cat_data:
  cat_data[i]=le.fit_transform(cat_data[i])
cat_data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,0,1,0,0,0,0,2
1,1,1,1,1,0,0,0
2,2,1,1,0,0,1,2
3,3,1,1,0,1,0,2
4,4,1,0,0,0,0,2
...,...,...,...,...,...,...,...
609,609,0,0,0,0,0,0
610,610,1,1,3,0,0,0
611,611,1,1,1,0,0,2
612,612,1,1,2,0,0,2


La colonne LoanId doit etre supprimer avant de commencer le traitement des donnes vue qu'elle n'a aucune utilite au niveau de notre model

In [31]:
#Supprimer loan_id
cat_data.drop('Loan_ID',axis=1,inplace=True)

Maintenant on a preparer nos donnes numeriques et categoriques, on doit les rassembler dans une seule base de donnees

In [32]:
#Concatener cat_data et num_data + specifier la trget
X=pd.concat([cat_data,num_data],axis=1)
Y=target

In [33]:
X

,Gender,Married,Dependents,Education,Self_Employed,Property_Area,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
0,1,0,0,0,0,2,5849.0,0.0,128.0,360.0,1.0
1,1,1,1,0,0,0,4583.0,1508.0,128.0,360.0,1.0
2,1,1,0,0,1,2,3000.0,0.0,66.0,360.0,1.0
3,1,1,0,1,0,2,2583.0,2358.0,120.0,360.0,1.0
4,1,0,0,0,0,2,6000.0,0.0,141.0,360.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,0,2900.0,0.0,71.0,360.0,1.0
610,1,1,3,0,0,0,4106.0,0.0,40.0,180.0,1.0
611,1,1,1,0,0,2,8072.0,240.0,253.0,360.0,1.0
612,1,1,2,0,0,2,7583.0,0.0,187.0,360.0,1.0
